In [1]:
import pandas as pd
import re

In [2]:
def column_types(df):
    col_types = []
    for col in df.columns:
        is_datetime = all(re.match(r'\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}', str(value)) for value in df[col])
        # Datetime format "YYYY-MM-DD HH:MM:SS"

        is_date = all(re.match(r'\d{4}-\d{2}-\d{2}', str(value)) for value in df[col])
        # Date format "YYYY-MM-DD"
        if is_datetime:
            col_types.append('DATETIME')
        elif is_date:
            col_types.append('DATE')
        else:
            dtype = df[col].dtype
            if dtype == 'int64' or dtype == 'int32':
                col_types.append('INT')
            elif dtype == 'float64' or dtype == 'float32':
                col_types.append('FLOAT')
            elif dtype == 'object':
                col_types.append('TEXT')
            else:
                col_types.append('TEXT')

    return col_types

In [3]:
def create_sql_query(table_name, columns, column_types):
    columns_with_types = [f'{col} {col_type}' for col, col_type in zip(columns, column_types)]
    create_query = f'CREATE TABLE {table_name} ({", ".join(columns_with_types)});'
    return create_query

In [4]:
def insert_sql_query(table_name, columns, df, column_types):
    insert_query = []
    for index, row in df.iterrows():
        formatted_values = [
            f"'{row[col]}'" if col_type in ['TEXT', 'DATETIME', 'DATE'] else str(row[col])
            for col, col_type in zip(columns, column_types)
        ]
        values_str = ', '.join(formatted_values)
        insert_sql = f'INSERT INTO {table_name} ({", ".join(columns)}) VALUES ({values_str});'
        insert_query.append(insert_sql)
    return insert_query

In [5]:
def create_dataframe(df, table_name):
    columns = df.columns
    column_types_list = column_types(df)
    create_query = create_sql_query(table_name, columns, column_types_list)
    insert_query = insert_sql_query(table_name, columns, df, column_types_list)
    print(create_query)
    print('\n'.join(insert_query))
    return create_query, insert_query

In [6]:
df = pd.read_csv('booth.csv')
df.head()

,booth_number,booth_price_level,booth_description,booth_type
0,1,A,First booth to the left as you enter the cente...,Standard
1,2,A,Second booth to the left as you enter the cent...,Standard
2,3,B,Third booth to the left as you enter the cente...,Small
3,4,C,First booth on left end of building - Small sq...,Small
4,5,C,Second booth on left end of building - Small ...,Small


In [7]:
create_query, insert_query = create_dataframe(df, 'booth')

CREATE TABLE booth (booth_number INT, booth_price_level TEXT, booth_description TEXT, booth_type TEXT);
INSERT INTO booth (booth_number, booth_price_level, booth_description, booth_type) VALUES (1, 'A', 'First booth to the left as you enter the center door', 'Standard');
INSERT INTO booth (booth_number, booth_price_level, booth_description, booth_type) VALUES (2, 'A', 'Second booth to the left as you enter the center door', 'Standard');
INSERT INTO booth (booth_number, booth_price_level, booth_description, booth_type) VALUES (3, 'B', 'Third booth to the left as you enter the center door - Small square table size', 'Small');
INSERT INTO booth (booth_number, booth_price_level, booth_description, booth_type) VALUES (4, 'C', 'First booth on left end of building - Small square table size', 'Small');
INSERT INTO booth (booth_number, booth_price_level, booth_description, booth_type) VALUES (5, 'C', 'Second booth on left end of building  - Small square table size', 'Small');
INSERT INTO booth 